# Industry training model 

行業部分，我們選用前兩個欄位 k_a08a_1、k_a08a_2 做斷字(單一個字)，當作input的feature，
接著建立LSTM模型，調整參數後儲存validation最高的數個模型，存成pickle之後在test預測時使用。參數主要是調整LSTM的unit數和dropout rate。

In [ ]:
import keras
from openpyxl import load_workbook
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional
from keras.optimizers import SGD, RMSprop
from keras import layers
from keras.models import Model
from keras import Input
import numpy as np
import jieba
import pickle
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

if __name__ == '__main__':
    wb = load_workbook('train.xlsx')
    flag = np.zeros((3167,64))
    ws = wb['Sheet1']
    cnt = 0
    corpus = []
    f = open('ind_real.csv')
    tmp = f.readlines()
    mp = np.zeros((100))
    for x in tmp:
        s = x.split(',')
        id1 = int(s[1].rstrip('\n'))
        if id1>100:
            continue
        mp[id1] = int(s[0])
    for row in ws.rows:
        if cnt==0:
            cnt +=1
            continue
        ls = []
        tmp_ans = []
        for cell in row:
            ls.append(cell.value)
        ind = int(ls[1])
        if ind > 100:
            continue
        ind = int(mp[ind])
        flag[cnt-1][ind] = 1
        tmp_list = []
        for x in range(len(ls[3])):
            tmp_list.append(ls[3][x])
        for x in range(len(ls[4])):
            tmp_list.append(ls[4][x])

        corpus.append(tmp_list)
        cnt+=1
    MAX_NUM_WORDS = 5000

    tokenizer = keras.preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS)
    tokenizer.fit_on_texts(corpus)

    x1_train = []
    for x in range(3167):
        x1_train.append(tokenizer.texts_to_sequences(corpus[x]))

    max_seq_len = max([len(seq) for seq in x1_train])

    x_train = np.zeros((3167,40))
    for x in range(3167):
        for y in range(len(x1_train[x])):
            ls = x1_train[x][y]
            if len(ls)==0:
                continue
            x_train[x][y] = ls[0]

    NUM_LSTM_UNITS = 256

    model = Sequential()
    model.add(layers.Embedding(MAX_NUM_WORDS, 1024))
    model.add(LSTM(256, dropout=0.1, recurrent_dropout=0.9))
#     model.add(Bidirectional(LSTM(256, dropout=0.1, recurrent_dropout=0.5, return_sequences=True)))
#     model.add(Bidirectional(LSTM(256, dropout=0.1, recurrent_dropout=0.5, return_sequences=False)))
    model.add(Activation('tanh'))

    model.add(Dense(units=64, activation='softmax'))

    with open('industry_tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    rms=keras.optimizers.RMSprop(lr=0.001)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=1,verbose=0,
                                mode='auto', min_delta=0.0001,
                                cooldown=0, min_lr=0.001)
    
    model.compile(optimizer=rms,loss='categorical_crossentropy',metrics=['accuracy'])
    earlystopping = EarlyStopping(monitor='val_acc', patience = 30, verbose=1, mode='max') 
    checkpoint = ModelCheckpoint(filepath='industry2.hdf5', verbose=1, save_best_only=True, \
                                save_weights_only=False, monitor='val_acc', mode='max')
    model.summary()
    model.fit(x_train,flag,batch_size=128,epochs=300,shuffle=True,verbose=1,validation_split=0.1, callbacks=[checkpoint,earlystopping,reduce_lr])



# Career training model

職業部分，我們選用後三個欄位 k_a08a_3、k_a08a_4、k_a08a_5 做斷字(單一個字)，當作input的feature，
接著建立LSTM模型，調整參數後儲存validation最高的數個模型，存成pickle之後在test預測時使用。參數主要是調整LSTM的unit數和dropout rate。

In [ ]:
import keras
from openpyxl import load_workbook
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional
from keras.optimizers import SGD  
from keras import layers
from keras.models import Model
from keras import Input
import numpy as np
import jieba
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import pickle

if __name__ == '__main__':
    wb = load_workbook('train.xlsx')
    flag = np.zeros((3200,120))
    ws = wb['Sheet1']
    cnt = 0
    corpus = []
    f = open('job_real.csv')
    tmp = f.readlines()
    mp = np.zeros((10000,))
    for x in tmp:
        s = x.split(',')
        id1 = int(s[1].rstrip('\n'))
        mp[id1] = int(s[0])
    for row in ws.rows:
        if cnt==0:
            cnt +=1
            continue
        ls = []
        tmp_ans = []
        for cell in row:
            ls.append(cell.value)

        tmp_list = []
        ind = int(ls[2])
        ind = int(mp[ind])
        flag[cnt-1][ind] = 1
        for x in range(len(ls[5])):
            tmp_list.append(ls[5][x])
        for x in range(len(ls[6])):
            tmp_list.append(ls[6][x])
        for x in range(len(ls[7])):
            tmp_list.append(ls[7][x])

        corpus.append(tmp_list)
        cnt+=1
    MAX_NUM_WORDS = 3000

    tokenizer = keras.preprocessing.text.Tokenizer(num_words=MAX_NUM_WORDS)
    tokenizer.fit_on_texts(corpus)

    x1_train = []
    for x in range(3200):
        x1_train.append(tokenizer.texts_to_sequences(corpus[x]))

    max_seq_len = max([len(seq) for seq in x1_train])
    print(max_seq_len)
    x_train = np.zeros((3200,65))
    for x in range(3200):
        for y in range(len(x1_train[x])):
            ls = x1_train[x][y]
            if len(ls)==0:
                continue
            x_train[x][y] = ls[0]

    model = Sequential()
    model.add(layers.Embedding(MAX_NUM_WORDS, 1024))
#     model.add(LSTM(256, dropout=0.1, recurrent_dropout=0.9))
    model.add(Bidirectional(LSTM(256, dropout=0.1, recurrent_dropout=0.5, return_sequences=True)))
    model.add(Bidirectional(LSTM(256, dropout=0.1, recurrent_dropout=0.5, return_sequences=False)))
    model.add(Activation('tanh'))

    model.add(Dense(units=120, activation='softmax'))

    with open('job_tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

#     sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    rms=keras.optimizers.RMSprop(lr=0.001)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=1,verbose=0,
                                mode='auto', min_delta=0.0001,
                                cooldown=0, min_lr=0.001)
    model.compile(optimizer=rms,loss='categorical_crossentropy',metrics=['accuracy'])
    model.summary()
    earlystopping = EarlyStopping(monitor='val_acc', patience = 50, verbose=1, mode='max') 
    checkpoint = ModelCheckpoint(filepath='career2.hdf5', verbose=1, save_best_only=True, \
                                save_weights_only=False, monitor='val_acc', mode='max')
    model.summary()
    model.fit(x_train,flag,batch_size=128,epochs=300,shuffle=True,verbose=1,validation_split=0.1, callbacks=[checkpoint,earlystopping,reduce_lr])



# Test 

利用前面 train 完後，validation 準確率較高的 10 個模型，利用ensemble方法，給予不同準確率的模型權重，再預測最有可能的職業及行業，結果會儲存在 ml.csv 中。

In [ ]:
import jieba
import numpy as np
from openpyxl import load_workbook
import pickle
import keras
from openpyxl import load_workbook
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation,LSTM
from keras.optimizers import SGD 
from keras import layers
from keras.models import Model
from keras import Input
from keras.models import load_model
if __name__ == '__main__':
    wb = load_workbook('test.xlsx')
    with open('job_tokenizer.pickle', 'rb') as handle:
        job_tokenizer = pickle.load(handle)
    with open('industry_tokenizer.pickle', 'rb') as handle:
        industry_tokenizer = pickle.load(handle)
    ws = wb['Sheet1']
    cnt = 0
    ind_model = load_model('industry20.hdf5')
    ind_model2 = load_model('industry8.hdf5')
    ind_model3 = load_model('industry14.hdf5')
    ind_model4 = load_model('industry19.hdf5')
    ind_model5 = load_model('industry15.hdf5')
    ind_model6 = load_model('industry12.hdf5')
    ind_model7 = load_model('industry7.hdf5')
    ind_model8 = load_model('industry16.hdf5')
    ind_model9 = load_model('industry18.hdf5')
    ind_model10 = load_model('industry17.hdf5')

    job_model = load_model('career11.hdf5')
    job_model2 = load_model('career8.hdf5')
    job_model3 = load_model('career9.hdf5')
    job_model4 = load_model('career12.hdf5')
    job_model5 = load_model('career6.hdf5')
    job_model6 = load_model('career10.hdf5')
    job_model7 = load_model('career7.hdf5')
    job_model8 = load_model('career3.hdf5')
    job_model9 = load_model('career2.hdf5')
    job_model10 = load_model('career5.hdf5')

    ou = open('ml.csv','w')
    f = open('job_real.csv','r')
    tmp = f.readlines()
    mp2 = np.zeros((200,))
    for x in tmp:
        s = x.split(',')
        #print(s)
        id1 = int(s[1].rstrip('\n'))
        id0 = int(s[0])
        print(id0,id1)
        mp2[id0] = id1
    mp1 = np.zeros((200,))
    f = open('ind_real.csv','r')
    tmp = f.readlines()
    for x in tmp:
        s = x.split(',')
        #print(s)
        id1 = int(s[1].rstrip('\n'))
        print(id1)

        id0 = int(s[0])
        mp1[id0] = id1
    print('x01,prediction',file=ou)
    for row in ws.rows:
        if cnt==0:
            cnt +=1
            continue
        ls = []
        tmp_ans = []

        for cell in row:
            ls.append(cell.value)
        ind = ls[0]

        tmp_list = []
        tmp_list2 = []
        cut = jieba.cut_for_search(ls[1])
        for x in range(len(ls[1])):
            tmp_list.append(ls[1][x])
        for x in range(len(ls[2])):
            tmp_list.append(ls[2][x])
        for x in range(len(ls[3])):
            tmp_list2.append(ls[3][x])
        for x in range(len(ls[4])):
            tmp_list2.append(ls[4][x])
        for x in range(len(ls[5])):
            tmp_list2.append(ls[5][x])
        st1 = industry_tokenizer.texts_to_sequences(tmp_list)
        st2 = job_tokenizer.texts_to_sequences(tmp_list2)
        inp = np.zeros((1,64))
        i = 0
        for x in st1:
            if len(x)!=0:
                inp[0,i] = x[0]
                i+=1
            else:
                i+=1
        #print(inp)
        '''model ensemble'''
        p1 = ind_model.predict(inp)
        p12 = ind_model2.predict(inp)
        p13 = ind_model3.predict(inp)
        p14 = ind_model4.predict(inp)
        p15 = ind_model5.predict(inp)
        p16 = ind_model6.predict(inp)
        p17 = ind_model7.predict(inp)
        p18 = ind_model8.predict(inp)
        p19 = ind_model9.predict(inp)
        p10 = ind_model10.predict(inp)
        p1=p1+p12+0.8*p13+0.8*p14+0.6*p15+0.6*p16+0.4*p17+0.4*p18+0.2*p19+0.2*p10

        res1  = np.argmax(p1)
        print(res1)
        inp = np.zeros((1,65))
        i = 0
        for x in st2:
            if len(x)!=0:
                inp[0,i] = x[0]
                i+=1
            else:
                i+=1
        #print(inp)
        '''model ensemble'''
        p2 = job_model.predict(inp)
        p22 = job_model2.predict(inp)
        p23 = job_model3.predict(inp)
        p24 = job_model4.predict(inp)
        p25 = job_model5.predict(inp)
        p26 = job_model6.predict(inp)
        p27 = job_model7.predict(inp)
        p28 = job_model8.predict(inp)
        p29 = job_model9.predict(inp)
        p20 = job_model10.predict(inp)
        p2=p2+p22+0.8*p23+0.8*p24+0.6*p25+0.6*p26+0.4*p27+0.4*p28+0.2*p29+0.2*p20

        res2  = np.argmax(p2)
        print(res2)
        print(str(ind)+'_a08a01,'+str(int(mp1[res1])),file=ou)
        print(str(ind)+'_a08a02,'+str(int(mp2[res2])),file=ou)

助教若需要執行，可以只執行test.py這個檔案就好，之前訓練完的模型已經存起來了(.hdf5檔)。training model分別是train_industry.py及train_career.py檔。我們最好的兩次預測結果在ml_good.csv及ml_goooood.csv這兩個檔案。

完整執行步驟為：train_industry.py > train_career.py > test.py